# fine tuning with together.AI

In [5]:
from together import Together
import os
import dotenv
dotenv.load_dotenv()

True

In [6]:
client=Together(api_key=os.getenv('TOGETHER_API_KEY'))

In [29]:
# check if API is working
stream=client.chat.completions.create(
    model=os.getenv('TOGETHERAI_MODEL_NAME'), #meta llama 3.1 7b
    # model='amruthpaiuni/Meta-Llama-3-8B-Instruct-llama-law-405B-Instruct-Turbo-43ad47fe',
    messages=[
        {
        'role':'user',
        'content':'what is life? what is its purpose?'
        }
    ],
    stream=True,
)
for chunk in stream:
    print(chunk.choices[0].delta.content or "",end="", flush=True)

The questions of "what is life?" and "what is its purpose?" have been debated and explored by philosophers, scientists, theologians, and thinkers across various disciplines for centuries. While there is no definitive answer, here's a comprehensive overview of the different perspectives:

**Biological Perspective:**
From a biological standpoint, life is a complex process that involves the organization of molecules, cells, and organisms. It is characterized by the ability to grow, reproduce, respond to stimuli, and maintain homeostasis. This perspective views life as a natural phenomenon that emerged on Earth through the process of evolution.

**Philosophical Perspective:**
Philosophers have offered various definitions of life, including:

1. **Vitalism:** The idea that life is a non-physical, essential force that animates living beings.
2. **Mechanistic:** The view that life is a complex machine that can be understood through its constituent parts and their interactions.
3. **Holistic:*

In [8]:
# create embeddings 
response=client.embeddings.create(
    model=os.getenv('TOGETHERAI_EMBEDDING_MODEL_NAME'),
    input='Amruth Pai, The man who started the AI Automation revolution',
)
print(response.data[0].embedding)

[-0.013148682, -0.068655774, 0.013548881, -0.014697075, -0.015767198, 0.021059586, 0.06448748, -0.08056574, 0.010573696, -0.014751823, 0.01213288, -0.08379289, 0.03079088, -0.011853566, 0.07861586, 0.069362216, -0.04998, -0.02392456, 0.015476803, -0.0007333028, -0.013277009, 0.017329935, -0.079270855, -0.040862862, 0.026545297, -0.03596333, -0.02750488, -0.030646235, 0.025378054, 0.04774166, -0.027231393, 0.044455796, -0.09527628, -0.04886891, -0.010372312, 0.04133453, 0.013665169, -0.033264294, 0.031922348, 0.06390309, -0.069591306, 0.00065552804, 0.028914468, -0.042982493, 0.03540486, 0.0022445964, -0.08069806, -0.0046245283, 0.10867481, -0.01209722, -0.013123243, -0.011249201, -0.044914443, 0.02096243, 0.055272054, -0.04973716, 0.03702042, 0.082806766, 0.05003372, -0.027347509, -0.021704955, -0.054740753, -0.014940327, -0.003401814, -0.0071875905, -0.098338276, 0.011742407, -0.060291838, 0.02775296, -0.059192613, -0.030467413, -0.01651141, 0.059013832, -0.0028378346, -0.05764156, -0

# fine tuning the model using instruction dataset 

In [9]:
# convert the json file to jsonl file 
import json
with open ('../data/laywer_response_instruct.json','r') as f:
    file=f.read()
# print(file)
parsed_json=json.loads(file)

In [19]:
import json

dataset = "laywer_response_instruct"
old_file_path = f"../data/{dataset}.json"
new_file_path = f"../data/Formatted{dataset}.jsonl"

# Load old format JSON data
with open(old_file_path, "r", encoding="utf-8") as old_file:
    old_data = json.load(old_file)

# Define Llama-3 prompt and system prompt
llama_format = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>
{user_question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{model_answer}<|eot_id|>
"""
formatted_data = []
system_prompt = "You're a lawyer assisting an iimigrant who needs legal assistance"

# Transform the data into the correct format and write it to a JSONL file
with open(new_file_path, "w", encoding="utf-8") as new_file:
    for piece in old_data:
        temp_data = {
            "text": llama_format.format(
                system_prompt=system_prompt,
                user_question=piece["instruction"],
                model_answer=piece["output"],
            )
        }
        new_file.write(json.dumps(temp_data))
        new_file.write("\n")


In [20]:
from together.utils import check_file

report = check_file(new_file_path)
print(report)
assert report["is_check_passed"] == True

{'is_check_passed': True, 'message': 'Checks passed', 'found': True, 'file_size': 7794626, 'utf8': True, 'line_type': True, 'text_field': True, 'key_value': True, 'min_samples': True, 'num_samples': 9241, 'load_json': True, 'filetype': 'jsonl'}


In [21]:
# upload the dataset for taining 
client=Together(api_key=os.getenv('TOGETHER_API_KEY'))
response=client.files.upload(file='../data/Formattedlaywer_response_instruct.jsonl')
filesID=response.model_dump()['id']

Uploading file Formattedlaywer_response_instruct.jsonl: 100%|██████████| 7.79M/7.79M [00:03<00:00, 2.34MB/s]


In [22]:
# verify the file upload 
file_metadata=client.files.retrieve(filesID)
print(file_metadata)

id='file-f29025b2-6e46-4772-a3d5-67ba34f1bbfd' object=<ObjectType.File: 'file'> created_at=1731246749 type=None purpose=<FilePurpose.FineTune: 'fine-tune'> filename='Formattedlaywer_response_instruct.jsonl' bytes=7794626 line_count=0 processed=True FileType='jsonl'


# start fine-tuning jobs

In [ ]:
resp=client.fine_tuning.create(
    suffix='llama3-law-8B-Instruct-Turbo3',
    model='meta-llama/Meta-Llama-3-8B-Instruct',
    training_file=filesID,
    n_epochs=4,
    batch_size=8,
    learning_rate=1e-5,
)



In [31]:
print(resp)

id='ft-7f393a1d-09f7-4cf0-afd4-5dd409b9b294' training_file='file-f29025b2-6e46-4772-a3d5-67ba34f1bbfd' validation_file='' model='meta-llama/Meta-Llama-3-8B-Instruct' output_name='amruthpaiuni/Meta-Llama-3-8B-Instruct-llama-law-405B-Instruct-Turbo2-fb896b74' adapter_output_name=None n_epochs=5 n_checkpoints=1 n_evals=0 batch_size=8 learning_rate=1e-05 warmup_ratio=0.0 eval_steps=0 training_type=FullTrainingType(type='Full') created_at='2024-11-10T14:28:56.735Z' updated_at='2024-11-10T14:28:56.735Z' status=<FinetuneJobStatus.STATUS_PENDING: 'pending'> job_id='' events=[FinetuneEvent(object=<ObjectType.FinetuneEvent: 'fine-tune-event'>, created_at='2024-11-10T14:28:56.735Z', level=<FinetuneEventLevels.NULL: ''>, message='Fine tune request created', type=<FinetuneEventType.JOB_PENDING: 'JOB_PENDING'>, param_count=0, token_count=0, wandb_url='', hash='', eval_token_count=0, total_steps=0, step=0, checkpoint_path='', model_path='', adapter_path='', training_offset=0)] token_count=0 param_cou